In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [38]:
tf.reset_default_graph()
learning_rate = 0.001
training_epochs = 20
batch_size = 100

class Model:
    
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            #dropout(keep_prob) rate 0.5~0.7 on training, but should be 1 for testing
            self.training = tf.placeholder(tf.bool)
            #input place holders
            self.X = tf.placeholder(tf.float32, [None,784])
            #img 28 * 28 *1(black/white)
            X_img = tf.reshape(self.X,[-1,28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
        # Conv layer 1
        #L1 ImgIn shape = (?,28,28,1)
        with tf.name_scope('ConvLayer1'):
            # Conv   ->    (?,28,28,32)
            # Pool   ->    (?,14,14,32)
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], 
                                     padding='SAME',activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], 
                                           padding='SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs=pool1, rate = 0.3, training=self.training)

        # Conv layer 2
        # L2 ImgIn shape = (?, 14, 14, 32)
        with tf.name_scope('ConvLayer2'):
            # Conv   ->    (?,14,14,64)
            # Pool   ->    (?,7,7,64)
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], 
                                     padding='SAME',activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], 
                                           padding='SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate = 0.3, training=self.training)

        with tf.name_scope('ConvLayer3'):
            # Conv   ->    (?,7,7,128)
            # Pool   ->    (?,4,4,128)
            # Reshape ->   (?,4 * 4 * 128) Flatten them for FC
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3], 
                                     padding='SAME',activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], 
                                           padding='SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate = 0.3, training=self.training)

        #Fully Connected layer1
        #FC 4X4X128 inputs -> 625 outputs
        with tf.name_scope('Fully_connected1'):
            flat = tf.reshape(dropout3,[-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)

        #Fully Connected layer2
        #Final FC 625 inputs -> 10 outputs
        with tf.name_scope('Fully_connected2'):
            self.logits = tf.layers.dense(inputs=dropout4, units=10)
           
        # cost/Loss func
        with tf.name_scope('cost'):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
    
        # optimizer    
        with tf.name_scope('optimizer'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        with tf.name_scope('accuarcuy'):
            correct_prediction = tf.equal(tf.arg_max(self.logits,1),tf.arg_max(self.Y,1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict = {self.X:x_test, self.training: training})
 
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict = {self.X:x_test, self.Y:y_test,self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X: x_data, self.Y:y_data,self.training: training})

In [39]:
with tf.Session() as sess:
    model = []
    num_models =2
    for m in range(num_models):
        model.append(Model(sess, 'model'+str(m)))
    
    sess.run(tf.global_variables_initializer())
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs")
    writer.add_graph(sess.graph)
    
    print('Learning started!')
    for epoch in range(training_epochs):
        avg_cost_list = np.zeros(len(model))
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            #train model
            for m_idx, m in enumerate(model):
                c,_ = m.train(batch_xs,batch_ys)
                avg_cost_list[m_idx] += c / total_batch
        print("Epoch:", '%04d'%(epoch+1), 'cost=',avg_cost_list)
    print('Learning Finished!')
    
    test_size = len(mnist.test.labels)
    predictions = np.zeros([test_size,10])
    for m_idx, m in enumerate(model):
        print(m_idx, 'Accuarcy:',m.get_accuracy(mnist.test.images,mnist.test.labels))
        p = m.predict(mnist.test.images)
        predictions += p
    ensemble_correct_prediction = tf.equal(tf.arg_max(predictions,1),tf.arg_max(mnist.test.labels,1))
    ensemble_accuarcy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
    print('Ensemble accuracy:',sess.run(ensemble_accuarcy))

Learning started!


KeyboardInterrupt: 